In [70]:
import pandas as pd 
import numpy as np

In [71]:
path = '../../supp_data/'

In [72]:
%%bash 

cd ../../supp_data

ls

avg_miles_covered_by_driver_per_dat.xlsx
curated_supp_data
daily_miles_by_state_county_tract.csv
data_dict_miles_per_household.xlsx
road_perc_accepted_miles15.xlsx
road_perc_accepted_miles16.xlsx
road_perc_accepted_miles17.xlsx
road_perc_accepted_miles18.xlsx
state_to_code.csv
weather_by_state.csv
ZIP-COUNTY-FIPS_2017-06.csv


In [73]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [74]:
# !python -m pip install "dask[complete]" 

## Loading all supp datasets

In [75]:
# acc_miles15 = pd.read_excel(path + 'road_perc_accepted_miles15.xlsx', index_col = 0)
# acc_miles16 = pd.read_excel(path + 'road_perc_accepted_miles16.xlsx', index_col = 0)
acc_miles17 = pd.read_excel(path + 'road_perc_accepted_miles17.xlsx', index_col = 0)
# acc_miles18 = pd.read_excel(path + 'road_perc_accepted_miles18.xlsx', index_col = 0)

# acc_miles = pd.concat([acc_miles15, acc_miles16, acc_miles17, acc_miles18])
acc_miles = acc_miles17

In [76]:
# for the sake of simplicity, I will data from 2017 only for acceptable miles. 2017 acceptable miles can predict both 2017(training) and 2018 (testing) 

In [77]:
weather = pd.read_csv(path + 'weather_by_state.csv', index_col = 0)

In [78]:
miles_covered = pd.read_csv(path + 'daily_miles_by_state_county_tract.csv', index_col = 0)

In [79]:
zip_county_conversion = pd.read_csv(path + 'ZIP-COUNTY-FIPS_2017-06.csv', index_col = 0)

In [80]:
state_to_code = pd.read_csv(path + 'state_to_code.csv', index_col = 0)

In [81]:
acc_miles.head()

,Total (miles),Acceptable (miles),Percent Acceptable,Urban (miles),Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Rural (miles),Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
NaN,Overall,Overall,Overall,Other principal arterials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other freeways and expressways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Minor Arterial,NaN,NaN,NaN,NaN,Major and Minor Collector,NaN,NaN,NaN,NaN,Interstate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other principal arterials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other freeways and expressways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Minor arterials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Major Collector,NaN,NaN,NaN,NaN,Interstate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,NaN,NaN,NaN,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-170,171-220,More than 220,Less than 60,60-94,95-170,171-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-170,171-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220
United States,748350,588340,0.786183,6629,15953,10049,8067,6380,4489,3586,9112,2826,4444,1822,1101,699,382,230,314,3115,13334,32793,13511,15291,1275,9019,32991,16938,25879,7194,6461,2218,1330,833,446,277,335,25688,36415,13049,6439,3391,1611,925,1102,2533,2549,723,335,170,84,42,39,17635,43819,21271,13117,8022,3908,2341,2881,20454,68159,107648,29771,25911,14756,9413,2422,1179,620,290,156,159
Alabama,24729,21301,0.861377,397,495,150,79,51,34,19,47,9,8,5,2,2,1,0,0,202,671,862,204,282,84,640,1550,537,817,246,82,34,26,18,12,7,4,848,873,165,63,30,16,9,10,0,0,0,0,0,0,0,0,890,2130,456,210,95,35,8,38,794,4234,4342,564,780,354,139,35,18,12,2,1,1
Alaska,3243,2514,0.775208,13,35,28,19,12,9,7,11,0,0,0,0,0,0,0,0,20,53,67,21,14,14,69,98,38,60,14,40,13,7,2,1,0,1,64,117,51,48,51,40,34,117,0,0,0,0,0,0,0,0,37,50,31,44,49,37,30,47,75,248,251,73,83,204,341,157,114,78,45,26,35


From this dataset, we just need the percentage acceptable miles. Our theory is; the higher the percentage, the safer the surface on tires. 

In [82]:
acc_miles.reset_index(inplace = True)

In [83]:
acc_miles.head()

,index,Total (miles),Acceptable (miles),Percent Acceptable,Urban (miles),Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Rural (miles),Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,NaN,Overall,Overall,Overall,Other principal arterials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other freeways and expressways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Minor Arterial,NaN,NaN,NaN,NaN,Major and Minor Collector,NaN,NaN,NaN,NaN,Interstate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other principal arterials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other freeways and expressways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Minor arterials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Major Collector,NaN,NaN,NaN,NaN,Interstate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,State,NaN,NaN,NaN,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-170,171-220,More than 220,Less than 60,60-94,95-170,171-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220,Less than 60,60-94,95-170,171-220,More than 220,Less than 60,60-94,95-119,120-144,145-170,171-194,195-220,More than 220
2,United States,748350,588340,0.786183,6629,15953,10049,8067,6380,4489,3586,9112,2826,4444,1822,1101,699,382,230,314,3115,13334,32793,13511,15291,1275,9019,32991,16938,25879,7194,6461,2218,1330,833,446,277,335,25688,36415,13049,6439,3391,1611,925,1102,2533,2549,723,335,170,84,42,39,17635,43819,21271,13117,8022,3908,2341,2881,20454,68159,107648,29771,25911,14756,9413,2422,1179,620,290,156,159
3,Alabama,24729,21301,0.861377,397,495,150,79,51,34,19,47,9,8,5,2,2,1,0,0,202,671,862,204,282,84,640,1550,537,817,246,82,34,26,18,12,7,4,848,873,165,63,30,16,9,10,0,0,0,0,0,0,0,0,890,2130,456,210,95,35,8,38,794,4234,4342,564,780,354,139,35,18,12,2,1,1
4,Alaska,3243,2514,0.775208,13,35,28,19,12,9,7,11,0,0,0,0,0,0,0,0,20,53,67,21,14,14,69,98,38,60,14,40,13,7,2,1,0,1,64,117,51,48,51,40,34,117,0,0,0,0,0,0,0,0,37,50,31,44,49,37,30,47,75,248,251,73,83,204,341,157,114,78,45,26,35


In [84]:
acc_miles = acc_miles[['index', 'Percent Acceptable']]

In [85]:
acc_miles = acc_miles.iloc[2:, :]

In [86]:
acc_miles.head()

,index,Percent Acceptable
2,United States,0.786183
3,Alabama,0.861377
4,Alaska,0.775208
5,Arizona,0.803321
6,Arkansas,0.912018


In [87]:
state_to_code.head()

,Abbrev,Code
State,,
Alabama,Ala.,AL
Alaska,Alaska,AK
Arizona,Ariz.,AZ
Arkansas,Ark.,AR
California,Calif.,CA


In [88]:
acc_miles = acc_miles.merge(state_to_code, left_on = 'index', right_index = True)
acc_miles.head()

,index,Percent Acceptable,Abbrev,Code
3,Alabama,0.861377,Ala.,AL
4,Alaska,0.775208,Alaska,AK
5,Arizona,0.803321,Ariz.,AZ
6,Arkansas,0.912018,Ark.,AR
7,California,0.550144,Calif.,CA


In [89]:
acc_miles_codes = acc_miles[['Code', 'Percent Acceptable']]

In [90]:
acc_miles_codes.head()

,Code,Percent Acceptable
3,AL,0.861377
4,AK,0.775208
5,AZ,0.803321
6,AR,0.912018
7,CA,0.550144


In [91]:
acc_miles_codes.to_csv(path + 'curated_supp_data/accept_miles.csv')

## Working on weather data

In [92]:
weather.head()

,name,station,year,month,avg_temp,avg_max_temp,avg_prcp,avg_snow_depth
state,,,,,,,,
NaN,PLATFORM NO. 62137,621370,2015,4,44.086364,45.522727,40.905000,NaN
NaN,PLATFORM NO. 62137,621370,2015,6,51.321053,52.978947,21.050526,NaN
PR,EUGENIO MARIA DE HOSTOS AIRPO,785145,2015,1,81.677778,84.000000,27.775000,NaN
PR,EUGENIO MARIA DE HOSTOS AIRPO,785145,2015,5,87.717647,672.770588,17.645294,NaN
PR,EUGENIO MARIA DE HOSTOS AIRPO,785145,2015,8,87.457143,797.221429,21.426429,NaN


In [93]:
weather.year.value_counts()

2017    33240
2015    32715
2016    32672
2018    30318
Name: year, dtype: int64

For now, I will not need the station granularity. Thus, I will group by State

In [94]:
weather_state = weather.groupby(['state', 'year', 'month']).agg({'avg_temp' : 'mean', 'avg_max_temp' : 'mean',
       'avg_prcp' : 'mean', 'avg_snow_depth' : 'mean'}).reset_index()

In [95]:
weather_state.head()

,state,year,month,avg_temp,avg_max_temp,avg_prcp,avg_snow_depth
0,AK,2015,1,17.971178,72.990438,2.379781,475.543727
1,AK,2015,2,20.759009,49.222243,3.012146,345.913970
2,AK,2015,3,22.334774,71.193844,2.818260,492.077742
3,AK,2015,4,31.860780,42.486386,3.655609,503.802364
4,AK,2015,5,46.103190,64.287331,2.700007,734.360541


In [96]:
weather_state.shape

(2536, 7)

In [97]:
weather_state.to_csv(path + 'curated_supp_data/weather_state.csv')

## MIles Covered 

In [98]:
miles_covered = miles_covered[['est_vmiles']]
miles_covered = miles_covered.dropna().reset_index()
miles_covered.head()

,geocode,est_vmiles
0,9001010400,31.51
1,9001010500,26.11
2,9001010600,22.49
3,9001010700,24.17
4,9001010800,30.94


In [99]:
miles_covered['STCOUNTYFP'] = miles_covered.geocode.astype(str).str[:5].astype(int)

In [100]:
miles_covered = miles_covered.groupby('STCOUNTYFP').agg({'est_vmiles': 'mean'}).reset_index()

In [101]:
miles_covered.head()

,STCOUNTYFP,est_vmiles
0,10001,45.302813
1,10003,39.024062
2,10005,46.419434
3,10010,47.821667
4,10030,48.096774


In [102]:
zip_county_conversion.head()

,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
ZIP,,,,
36003,Autauga County,AL,1001,H1
36006,Autauga County,AL,1001,H1
36067,Autauga County,AL,1001,H1
36066,Autauga County,AL,1001,H1
36703,Autauga County,AL,1001,H1


In [103]:
zip_county_conversion = zip_county_conversion.reset_index()[['ZIP', 'STCOUNTYFP']]

In [104]:
zip_county_conversion.head()

,ZIP,STCOUNTYFP
0,36003,1001
1,36006,1001
2,36067,1001
3,36066,1001
4,36703,1001


In [105]:
zip_county_conversion = zip_county_conversion.merge(miles_covered, left_on = 'STCOUNTYFP', right_on = 'STCOUNTYFP', how = 'inner')

In [106]:
zip_county_conversion = zip_county_conversion[['ZIP', 'est_vmiles']]

In [107]:
zip_county_conversion.head()

,ZIP,est_vmiles
0,19904,45.302813
1,19980,45.302813
2,19952,45.302813
3,19962,45.302813
4,19901,45.302813


In [108]:
zip_county_conversion.to_csv(path + 'curated_supp_data/zip_county.csv')